In [ ]:
import logging.config
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import os
import shutil
import json
import pyodbc
import subprocess
import datetime as dt
import logging
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
Error_List = {
    "Intacct_GLAccount": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    },    "Intacct_DEPARTMENT": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    },    "Intacct_Project": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    },    "Intacct_GLTransaction": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    },    "GLBUDGETITEM": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    },    "GLBUDGETITEM": {
        "ErrorCount": 0,
        "Error List": [],
        "ErrorFiles": []
    }
}


class get_Intacct_data():
    def __init__(self, entity, sql_table_name, sort_col, full_refresh, li_cols, **kwargs):

        self.error_message = []
        self.sql_server_name = kwargs.get('sql_server_name')
        self.sql_database_name = kwargs.get('sql_database_name')
        self.receive_email = kwargs.get('emailaddress')
        self.sql_username = kwargs.get('sql_username')
        self.sql_password = kwargs.get('sql_password')
        self.sender_id = kwargs.get('sender_id')
        self.sender_password = kwargs.get('sender_password')
        self.company_id = kwargs.get('company_id')
        self.user_id = kwargs.get('user_id')
        self.user_password = kwargs.get('user_password')
        self.sql_table_name = sql_table_name
        self.sort_col = sort_col
        self.entity = entity
        self.full_refresh = full_refresh
        self.li_cols = li_cols
        self.url = "https://api.intacct.com/ia/xml/xmlgw.phtml"
        self.headers = {"Content-Type": "application/xml"}
        logging.basicConfig(filename='log.log', level=logging.INFO,
                            filemode='w', format='%(asctime)s:%(levelname)s:%(message)s')
        logging.info('_________Starting Proccess________')

    def query_list(self):
        query_cols = ''
        for i in self.li_cols:
            query_cols += f'<field>{i}</field>'
        return query_cols



    def run_sql_query(self, Mode):
        driver = '{ODBC Driver 17 for SQL Server}'
        try:
            connection = pyodbc.connect(
                f'DRIVER={driver};SERVER={self.sql_server_name};DATABASE={self.sql_database_name};UID={self.sql_username};PWD={self.sql_password}'
            )
            logging.info(f"Checking data in {self.sql_table_name}")
            cursor = connection.cursor()
            if Mode == 'select':
                query = f"SELECT COUNT(*) FROM {self.sql_table_name}"
                query2 = f"SELECT MAX(WHENMODIFIED) FROM {self.sql_table_name}"
                cursor.execute(query2)
                results2 = cursor.fetchall()
                for row in results2:
                    self.lastdate = row[0]
                cursor.execute(query)
                results = cursor.fetchall()
                for row in results:
                    return int(row[0])
            if Mode == 'Full_refresh':
                query = f"DELETE {self.sql_table_name}"
                cursor.execute(query)
                cursor.commit()
                return None
            if Mode == 'procedure':
                query = f"EXECUTE Processing_Data ?"
                cursor.execute(query, (self.sql_table_name,))
                cursor.commit()
                return None
            cursor.close()
            connection.close()
        except Exception as e:
            # pull error message to list
            Error_List[self.sql_table_name]["ErrorCount"] += 1
            Error_List[self.sql_table_name]["Error List"].append(str(e))
            logging.error(str(e))
            return -1

    def get_data(self, offset, last_date):
        if not last_date:
            last_date = dt.datetime.strptime(
                '1900-1-1 00:0:0', '%Y-%m-%d %H:%M:%S')
        if self.full_refresh != 1:
            temp_payload = f"""<orderby>
                                <order>
                                <field>{self.sort_col}</field>
                                <ascending/>
                                </order>
                            </orderby>
                            <filter>
                                <greaterthanorequalto>
                                    <field>{self.sort_col}</field>
                                        <value>{last_date.month}/{last_date.day}/{last_date.year} {last_date.hour}:{last_date.minute}:{last_date.second}</value>
                                </greaterthanorequalto>
                            </filter>"""
        else:
            temp_payload = ""
        xml_payload = f"""
        <request>
            <control>
                <senderid>{self.sender_id}</senderid>
                <password>{self.sender_password}</password>
                <controlid>control_test</controlid>
                <uniqueid>false</uniqueid>
                <dtdversion>3.0</dtdversion>
            </control>
            <operation>
                <authentication>
                    <login>
                        <userid>{self.user_id}</userid>
                        <companyid>{self.company_id}</companyid>
                        <password>{self.user_password}</password>
                    </login>
                </authentication>
                <content>
                    <function controlid="query_get_data">
                        <query>
                            <object>{self.entity}</object>
                            <select>
                                {self.query_list()}
                            </select>
                            <pagesize>1000</pagesize>
                            {temp_payload}
                            <offset>{offset}</offset>
                        </query>
                    </function>
                </content>
            </operation>
        </request>
        """
        if self.full_refresh == 1:
            logging.info(
                f'Get data of {self.entity} with modified that is on equal or after {last_date.month}/{last_date.day}/{last_date.year} {last_date.hour}:{last_date.minute}:{last_date.second}')
        # else:
        #     logging.info(f'Deleting table {self.sql_table_name}')
        return xml_payload

    def pull_data_to_sql(self, n, preflex):

        file_path = os.path.join(self.entity, f"{n}.csv")
        error_log = f"{file_path}_error.log"
        bcp_command = [
            "bcp",
            f"dbo.{self.sql_table_name}{preflex}",
            # "dbo.Intacct_GLTransaction_temp",
            "in",
            file_path,
            "-S", self.sql_server_name,
            "-d", self.sql_database_name,
            "-c",
            "-t", ",",
            "-U", self.sql_username,
            "-P", self.sql_password,
            "-e", error_log
        ]
        try:

            logging.info(f"Processing file: {n}.csv")
            result = subprocess.run(
                bcp_command, capture_output=True, text=True)
            if result.returncode == 0:
                logging.info(f"Successfully imported {n}.csv!")
                os.remove(error_log)
            else:
                logging.error(
                    f"Failed to import {n}.csv. Check the error log: {error_log}")
                Error_List[self.sql_table_name]["ErrorCount"] += 1
                Error_List[self.sql_table_name]["ErrorFiles"].append(error_log)

        except Exception as e:
            logging.error(f"An error occurred while processing {n}.csv: {e}")
            Error_List[self.sql_table_name]["ErrorCount"] += 1
            Error_List[self.sql_table_name]["Error List"].append(str(e))
        return None

    def run(self):
        self.lastdate = dt.datetime.strptime(
            '1900-1-1 00:0:0', '%Y-%m-%d %H:%M:%S')
        if self.full_refresh == 1:
            self.run_sql_query(Mode='Full_refresh')
            n_row_exist = 0
        else:
            n_row_exist = self.run_sql_query(Mode='select')
        match n_row_exist:
            case 0:
                logging.info(
                    f"This first running, please wait for pull data to {self.sql_table_name}")
            case default:
                logging.info(
                    f'Preparing to update data for {self.sql_table_name}')
        if n_row_exist != -1:
            offset = 0
            n = 0
            numremaining = 1
            if os.path.exists(self.entity):
                shutil.rmtree(self.entity)
                logging.info(f"The folder '{self.entity}' has been deleted.")
            os.makedirs(self.entity)
            idx = n_row_exist + 1
            while numremaining > 0:
                try:
                    D = {}
                    for i in self.li_cols:
                        D[i] = []
                    # Code that might raise an exception
                    response = requests.post(self.url, data=self.get_data(
                        offset, self.lastdate), headers=self.headers, timeout=60)

                    response.raise_for_status()  # Will raise an HTTPError for bad responses (4xx and 5xx)
                    if response.status_code == 200:
                        offset += 1000

                        n += 1
                        root = ET.fromstring(response.text).find('.//data')
                        for child in root.findall(self.entity):
                            for i in self.li_cols:
                                # D[i].append(child.find(i).text)
                                text_value = child.find(i).text
                                D[i].append(text_value.replace(
                                    ',', '') if text_value else '')
                        df = pd.DataFrame(D)
                        if df.shape[0] > 0:
                            df.index += idx
                            df['ETLUpdatedDateTime'] = dt.datetime.now().strftime(
                                "%#m/%#d/%Y  %#I:%M:%S %p")
                            df.to_csv(fr'{self.entity}\{n}.csv',
                                      index=True, header=False)
                            if n_row_exist > 0:

                                self.pull_data_to_sql(n, '_temp')
                            else:
                                self.pull_data_to_sql(n, '')
                        else:
                            logging.info('No data to update')
                        numremaining = int(root.get('numremaining'))
                        idx += 1000

                    else:
                        break
                except requests.exceptions.RequestException as e:
                    error_message = str(e)
                    # pull error message to list
                    logging.error(f"An error occurred while processing {n}.csv: {e}")
                    Error_List[self.sql_table_name]["ErrorCount"] += 1
                    logging.error(f"An error occurred: {error_message}")
                    numremaining = -1
                    break
                except Exception as e:
                    # pull error message to list
                    logging.error(f"An error occurred while processing {n}.csv: {e}")
                    Error_List[self.sql_table_name]["ErrorCount"] += 1
                    error_message = str(e)
                    logging.error(
                        f"An unexpected error occurred: {error_message}")
                    numremaining = -1
                    break
            if n_row_exist > 0:
                self.run_sql_query(Mode='procedure')
            
        return None


class Processing_Intacct():

    def __init__(self,):
        with open(r"config.json", "r") as file:
            self.data = json.load(file)
    def send_email_with_error(self, list_file, HTMLBody):
        smtp_server = "smtp.office365.com"
        smtp_port = 587
        sender_email = "rpa@email"
        sender_password = "rpaemailpass"

        # Create email message
        msg = MIMEMultipart()
        msg["From"] = sender_email
        msg["To"] =  ", ".join(self.data['D']['emailaddress']) 
        msg["Subject"] = f"Intacct Sync Status"

        body = HTMLBody
        msg.attach(MIMEText(body, "plain"))

        for file_path in list_file:
            if os.path.exists(file_path):
                table_name = os.path.basename(os.path.dirname(file_path))
                old_file_name = os.path.basename(file_path)
                new_file_name = f"{table_name}_{old_file_name}"
                new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)

                os.rename(file_path, new_file_path)

                with open(new_file_path, "rb") as attachment:
                    part = MIMEBase("application", "octet-stream")
                    part.set_payload(attachment.read())
                    encoders.encode_base64(part)
                    part.add_header("Content-Disposition", f'attachment; filename="{new_file_name}"')
                    msg.attach(part)
            else:
                print(f"File not found: {file_path}")

        # Send email
        try:
            server = smtplib.SMTP(smtp_server, smtp_port)
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, self.data['D']['emailaddress'], msg.as_string())
            server.quit()
        except Exception as e:
            print(f"Error: {e}")
    def run(self):
        total_error_count = 0
        HTMLNotify = ""
        HTMLFiles = []
        kwargs_load = self.data['D']
        for i in self.data['function_calls']:
            get_Intacct_data(i['table'], i['destination_table'], sort_col=i['sort_col'], full_refresh=i['full_refresh'], li_cols=i['li_cols'], **kwargs_load
                             ).run()
        # self.send_email_with_error(self.error_files, self.entity)
            total_error_count += Error_List[i["destination_table"]]["ErrorCount"]
        if total_error_count == 0:
            HTMLNotify = "Successfully pulled data from Intacct to SQL Cloud for the following tables: Intacct_GLAccount, Intacct_DEPARTMENT, Intacct_Project, Intacct_GLTransaction, GLBUDGETITEM, GLBUDGETITEM"
        else:
            for i in self.data['function_calls']:
                table_name = i["destination_table"]
                if Error_List[i["destination_table"]]["ErrorCount"] > 0:
                    HTMLNotify += f"❌ {table_name}: Failed with errors:\n"
                    for error_msg in Error_List[table_name]["Error List"]:
                        HTMLNotify += f"    ⚫ {error_msg}\n"
                    
                    if "ErrorFiles" in Error_List[table_name]:  
                        for error_file in Error_List[table_name]["ErrorFiles"]:
                            HTMLFiles.append(error_file)
                else:
                    HTMLNotify = HTMLNotify + f"✔ {table_name}: Successfully pulled.\n"
        
        self.send_email_with_error(HTMLFiles, HTMLNotify)

Test = Processing_Intacct().run()
